In [ ]:
import requests

# function for retrieving coordinates from address
def get_coords(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": "KakaoAK fabb38e7995e3d45142f6c4ed8a51698"}
    params = {"query": address}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get('documents'):
            x = data['documents'][0].get('x')
            y = data['documents'][0].get('y')
            if x is not None and y is not None:
                return [float(y), float(x)]
        return None
    except Exception as e:
        return None

In [ ]:
# if 도로명 value exists, concat 시군구, 도로명 values and retrieve coordinates
unique_road = unique_road.with_columns(
    pl.when(pl.col("도로명").is_not_null())
    .then(
        pl.concat_str(
            [pl.col("시군구"), pl.lit(" "), pl.col("도로명")]
        ).map_elements(
            get_coords,
            return_dtype=pl.List(pl.Float64)
        )
    )
    .otherwise(
        None
    )
    .alias("coords")
)

# extract lat and long
unique_road = unique_road.with_columns([
    pl.col('coords').list.get(1, null_on_oob=True).alias('long'),
    pl.col('coords').list.get(0, null_on_oob=True).alias('lat')
])

# if lat/long is null, concat 시군구, 번지 values and retrieve coordinates
unique_road = unique_road.with_columns(
    pl.when(pl.col('long').is_null()).
    then(
        pl.concat_str(
            [pl.col("시군구"), pl.lit(" "), pl.col("번지")]
        ).map_elements(
            get_coords,
            return_dtype=pl.List(pl.Float64)
        )
    )
    .otherwise(
        None
    )
    .alias('coords2')
)

In [ ]:
# if values in coords doesn't exist, use coords2 and assign new column new_coords
unique_road = unique_road.with_columns(
    pl.when(pl.col("coords").list.len() > 0)
      .then(pl.col("coords"))
      .otherwise(pl.col("coords2"))
      .alias("new_coords")
)

coordinates = unique_road.select(['시군구','도로명','번지','new_coords'])

coordinates = coordinates.with_columns([
    pl.col('new_coords').list.get(1, null_on_oob=True).alias('long'),
    pl.col('new_coords').list.get(0, null_on_oob=True).alias('lat')
])

coordinates = coordinates.select(pl.exclude('new_coords'))

main = main.join(coordinates.select(['시군구','도로명','번지','lat','long']),on=['시군구','도로명','번지'],how='left')
test = test.join(coordinates.select(['시군구','도로명','번지','lat','long']),on=['시군구','도로명','번지'],how='left')